# PyPort Blueprint and Entity Management

This notebook demonstrates how to manage blueprints and entities using the PyPort library.

## Setup

First, we need to install the PyPort library if it's not already installed:

In [ ]:
# Uncomment and run this cell if you need to install PyPort
# !pip install pyport

Next, we'll import the necessary modules and set up our API credentials:

In [ ]:
import os
import json
import uuid
from typing import Dict, Any, List
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Import PyPort
from pyport import PortClient
from pyport.exceptions import PortApiError

# Helper function to pretty-print JSON
def print_json(data: Dict[str, Any]) -> None:
    """Print JSON data in a readable format."""
    print(json.dumps(data, indent=2))

Set your Port API credentials. You can either set them as environment variables or directly in this notebook:

In [ ]:
# Option 1: Get credentials from environment variables
client_id = os.environ.get('PORT_CLIENT_ID')
client_secret = os.environ.get('PORT_CLIENT_SECRET')

# Option 2: Set credentials directly (not recommended for shared notebooks)
# client_id = "your-client-id"
# client_secret = "your-client-secret"

# Check if credentials are set
if not client_id or not client_secret:
    print("Warning: PORT_CLIENT_ID and PORT_CLIENT_SECRET environment variables are not set.")
    print("Please set these variables or provide them directly in the notebook.")
else:
    print("✅ API credentials are set!")

## Initializing the Client

Now, let's initialize the Port client:

In [ ]:
# Initialize the Port client
client = PortClient(
    client_id=client_id,
    client_secret=client_secret
)

print("✅ Client initialized successfully!")

## Creating a Blueprint

Let's create a new blueprint for a microservice:

In [ ]:
# Generate a unique identifier for our test blueprint
test_blueprint_id = f"test-microservice-{uuid.uuid4().hex[:8]}"
print(f"Using test blueprint identifier: {test_blueprint_id}")

# Define the blueprint data
blueprint_data = {
    "identifier": test_blueprint_id,
    "title": "Test Microservice",
    "icon": "Microservice",
    "description": "A test microservice blueprint created in the Jupyter notebook",
    "schema": {
        "properties": {
            "language": {
                "type": "string",
                "title": "Language",
                "enum": ["Python", "JavaScript", "Java", "Go", "Ruby"],
                "enumColors": {
                    "Python": "blue",
                    "JavaScript": "yellow",
                    "Java": "red",
                    "Go": "lightBlue",
                    "Ruby": "darkRed"
                }
            },
            "version": {
                "type": "string",
                "title": "Version",
                "default": "1.0.0"
            },
            "repository": {
                "type": "string",
                "title": "Repository URL",
                "format": "url"
            },
            "owner": {
                "type": "string",
                "title": "Owner"
            },
            "status": {
                "type": "string",
                "title": "Status",
                "enum": ["Active", "Deprecated", "In Development"],
                "enumColors": {
                    "Active": "green",
                    "Deprecated": "red",
                    "In Development": "yellow"
                },
                "default": "In Development"
            },
            "tags": {
                "type": "array",
                "title": "Tags",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": ["language", "version", "owner"]
    },
    "calculationProperties": {
        "languageIcon": {
            "title": "Language Icon",
            "calculation": "return `https://cdn.example.com/icons/${entity.properties.language.toLowerCase()}.png`",
            "type": "string",
            "format": "url"
        }
    },
    "relations": {}
}

try:
    # Create the blueprint
    new_blueprint = client.blueprints.create_blueprint(blueprint_data)
    print("✅ Blueprint created successfully!")
    
    # Display the created blueprint
    print("\nCreated Blueprint:")
    display(HTML(f"<b>Identifier:</b> {new_blueprint['identifier']}<br>"
                 f"<b>Title:</b> {new_blueprint['title']}<br>"
                 f"<b>Description:</b> {new_blueprint['description']}<br>"
                 f"<b>Created At:</b> {new_blueprint['createdAt']}"))
    
    # Display the blueprint properties
    print("\nBlueprint Properties:")
    properties = new_blueprint["schema"]["properties"]
    property_data = []
    for prop_name, prop_details in properties.items():
        property_data.append({
            "name": prop_name,
            "title": prop_details.get("title", prop_name),
            "type": prop_details.get("type", "unknown"),
            "required": prop_name in new_blueprint["schema"].get("required", [])
        })
    
    # Create and display DataFrame
    property_df = pd.DataFrame(property_data)
    display(property_df)
except PortApiError as e:
    print(f"❌ Error creating blueprint: {e}")

## Creating Entities

Now, let's create some entities based on our blueprint:

In [ ]:
# Define entity data for multiple entities
entity_data_list = [
    {
        "identifier": f"payment-service-{uuid.uuid4().hex[:8]}",
        "title": "Payment Service",
        "properties": {
            "language": "Python",
            "version": "1.2.0",
            "repository": "https://github.com/example/payment-service",
            "owner": "Team Finance",
            "status": "Active",
            "tags": ["payment", "finance", "core"]
        }
    },
    {
        "identifier": f"user-service-{uuid.uuid4().hex[:8]}",
        "title": "User Service",
        "properties": {
            "language": "Java",
            "version": "2.1.0",
            "repository": "https://github.com/example/user-service",
            "owner": "Team Identity",
            "status": "Active",
            "tags": ["user", "identity", "core"]
        }
    },
    {
        "identifier": f"notification-service-{uuid.uuid4().hex[:8]}",
        "title": "Notification Service",
        "properties": {
            "language": "Go",
            "version": "0.9.0",
            "repository": "https://github.com/example/notification-service",
            "owner": "Team Communications",
            "status": "In Development",
            "tags": ["notification", "communication"]
        }
    }
]

# Store entity IDs for later use
created_entity_ids = []

# Create each entity
for entity_data in entity_data_list:
    try:
        # Create the entity
        entity = client.entities.create_entity(test_blueprint_id, entity_data)
        print(f"✅ Entity '{entity['title']}' created successfully!")
        created_entity_ids.append(entity["identifier"])
    except PortApiError as e:
        print(f"❌ Error creating entity '{entity_data['title']}': {e}")

print(f"\nCreated {len(created_entity_ids)} entities!")

## Getting Entities

Let's get all entities for our blueprint and display them in a table:

In [ ]:
try:
    # Get all entities for our blueprint
    entities = client.entities.get_entities(test_blueprint_id)
    print(f"✅ Retrieved {len(entities)} entities for blueprint '{test_blueprint_id}'!")
    
    # Extract entity data for display
    entity_display_data = []
    for entity in entities:
        entity_display_data.append({
            "identifier": entity["identifier"],
            "title": entity["title"],
            "language": entity["properties"].get("language", ""),
            "version": entity["properties"].get("version", ""),
            "owner": entity["properties"].get("owner", ""),
            "status": entity["properties"].get("status", ""),
            "created_at": entity["createdAt"]
        })
    
    # Create and display DataFrame
    entity_df = pd.DataFrame(entity_display_data)
    display(entity_df)
except PortApiError as e:
    print(f"❌ Error retrieving entities: {e}")

## Updating an Entity

Let's update one of the entities we created:

In [ ]:
if created_entity_ids:
    # Get the first entity ID
    entity_id_to_update = created_entity_ids[0]
    
    try:
        # Get the current entity data
        entity = client.entities.get_entity(test_blueprint_id, entity_id_to_update)
        print(f"Current entity: {entity['title']} (Status: {entity['properties'].get('status', 'Unknown')})")
        
        # Define the update data
        update_data = {
            "title": f"{entity['title']} (Updated)",
            "properties": {
                "version": "2.0.0",  # Update version
                "status": "Deprecated"  # Change status
            }
        }
        
        # Update the entity
        updated_entity = client.entities.update_entity(test_blueprint_id, entity_id_to_update, update_data)
        print(f"✅ Entity updated successfully!")
        
        # Display the updated entity
        print(f"Updated entity: {updated_entity['title']} (Status: {updated_entity['properties'].get('status', 'Unknown')})")
        
        # Show before and after comparison
        comparison_data = [
            {
                "field": "Title",
                "before": entity["title"],
                "after": updated_entity["title"]
            },
            {
                "field": "Version",
                "before": entity["properties"].get("version", ""),
                "after": updated_entity["properties"].get("version", "")
            },
            {
                "field": "Status",
                "before": entity["properties"].get("status", ""),
                "after": updated_entity["properties"].get("status", "")
            }
        ]
        
        # Create and display DataFrame
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df)
    except PortApiError as e:
        print(f"❌ Error updating entity: {e}")
else:
    print("No entities available to update.")

## Searching for Entities

Let's search for entities based on specific criteria:

In [ ]:
try:
    # Define search query for active entities
    search_query = {
        "blueprint": test_blueprint_id,
        "properties": {
            "status": "Active"
        }
    }
    
    # Search for entities
    active_entities = client.entities.search_entities(search_query)
    print(f"✅ Found {len(active_entities)} active entities!")
    
    # Extract entity data for display
    if active_entities:
        active_entity_data = []
        for entity in active_entities:
            active_entity_data.append({
                "identifier": entity["identifier"],
                "title": entity["title"],
                "language": entity["properties"].get("language", ""),
                "owner": entity["properties"].get("owner", ""),
                "status": entity["properties"].get("status", "")
            })
        
        # Create and display DataFrame
        active_df = pd.DataFrame(active_entity_data)
        display(active_df)
    else:
        print("No active entities found.")
    
    # Define search query for entities with specific tag
    search_query = {
        "blueprint": test_blueprint_id,
        "properties": {
            "tags": {
                "$contains": "core"
            }
        }
    }
    
    # Search for entities
    core_entities = client.entities.search_entities(search_query)
    print(f"\n✅ Found {len(core_entities)} entities with 'core' tag!")
    
    # Extract entity data for display
    if core_entities:
        core_entity_data = []
        for entity in core_entities:
            core_entity_data.append({
                "identifier": entity["identifier"],
                "title": entity["title"],
                "language": entity["properties"].get("language", ""),
                "tags": ", ".join(entity["properties"].get("tags", []))
            })
        
        # Create and display DataFrame
        core_df = pd.DataFrame(core_entity_data)
        display(core_df)
    else:
        print("No entities with 'core' tag found.")
except PortApiError as e:
    print(f"❌ Error searching for entities: {e}")

## Visualizing Entity Data

Let's create some visualizations of our entity data:

In [ ]:
try:
    # Get all entities for our blueprint
    entities = client.entities.get_entities(test_blueprint_id)
    
    if entities:
        # Count entities by language
        language_counts = {}
        for entity in entities:
            language = entity["properties"].get("language", "Unknown")
            language_counts[language] = language_counts.get(language, 0) + 1
        
        # Create pie chart for language distribution
        plt.figure(figsize=(8, 8))
        plt.pie(language_counts.values(), labels=language_counts.keys(), autopct='%1.1f%%', startangle=90)
        plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
        plt.title("Entity Distribution by Language")
        plt.show()
        
        # Count entities by status
        status_counts = {}
        for entity in entities:
            status = entity["properties"].get("status", "Unknown")
            status_counts[status] = status_counts.get(status, 0) + 1
        
        # Create bar chart for status distribution
        plt.figure(figsize=(10, 6))
        plt.bar(status_counts.keys(), status_counts.values())
        plt.xlabel("Status")
        plt.ylabel("Number of Entities")
        plt.title("Entity Distribution by Status")
        plt.show()
    else:
        print("No entities available for visualization.")
except PortApiError as e:
    print(f"❌ Error creating visualizations: {e}")

## Deleting Entities and Blueprint

Finally, let's clean up by deleting the entities and blueprint we created:

In [ ]:
# Delete entities
for entity_id in created_entity_ids:
    try:
        success = client.entities.delete_entity(test_blueprint_id, entity_id)
        if success:
            print(f"✅ Entity '{entity_id}' deleted successfully!")
        else:
            print(f"❌ Failed to delete entity '{entity_id}'.")
    except PortApiError as e:
        print(f"❌ Error deleting entity '{entity_id}': {e}")

# Delete blueprint
try:
    success = client.blueprints.delete_blueprint(test_blueprint_id)
    if success:
        print(f"\n✅ Blueprint '{test_blueprint_id}' deleted successfully!")
    else:
        print(f"\n❌ Failed to delete blueprint '{test_blueprint_id}'.")
except PortApiError as e:
    print(f"\n❌ Error deleting blueprint '{test_blueprint_id}': {e}")

## Summary

In this notebook, we've demonstrated how to manage blueprints and entities using the PyPort library:

1. Creating a blueprint with properties, required fields, and calculation properties
2. Creating entities based on the blueprint
3. Getting and displaying entities in a table
4. Updating an entity and comparing before/after changes
5. Searching for entities based on specific criteria
6. Visualizing entity data with charts
7. Deleting entities and blueprints

These operations form the foundation for managing your resources in Port through the PyPort library.